In [282]:
import pandas as pd
import numpy as np
from autofeat import FeatureSelector
from sklearn.feature_selection import VarianceThreshold
from sklearn.impute import KNNImputer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.ensemble import IsolationForest

In [283]:
raw_dataset_x = pd.read_csv('X_train.csv',index_col=['id'])
X = raw_dataset_x.copy()
X.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x822,x823,x824,x825,x826,x827,x828,x829,x830,x831
id,,,,,,,,,,,,,,,,,,,,,
0.0,14168.823171,10514.380717,3316.149698,94230.695124,102.386606,92.677127,11108.748199,10866.505510,10837.622093,10.227734,...,NaN,12352.094085,846.014651,105.132144,102.112809,2090.004260,2.691845,1234.374109,1000.784475,9285.751272
1.0,17757.037554,NaN,4101.016273,92959.527633,NaN,99.855168,10013.959449,10826.607494,10076.101597,11.436970,...,NaN,16198.071494,776.084467,106.385590,103.472030,2474.051881,2.287976,NaN,1012.626705,11750.284764
2.0,14226.656663,11029.642499,NaN,124055.600561,100.542483,92.860892,NaN,10492.342868,NaN,10.810076,...,10329.704431,13976.063780,737.040332,103.671234,109.458246,2656.083281,2.843706,888.353607,1048.810385,9553.922728
3.0,8766.012436,7384.202998,2147.308418,100157.719990,104.855061,101.929026,10050.049932,10499.521099,10525.030989,10.092109,...,10008.251395,6212.127347,329.044233,105.084488,104.858546,1097.785204,2.732257,927.752967,1048.357330,NaN
4.0,13801.016418,13269.493652,3408.316953,92048.527786,103.759758,95.789235,9667.353978,10750.783106,10618.800750,12.006773,...,10095.782015,13772.061493,NaN,NaN,100.369834,2693.053231,2.702908,1471.354073,1071.284484,9423.533063


In [284]:
raw_dataset_y = pd.read_csv('y_train.csv',index_col=['id'])
y = raw_dataset_y.copy()
y.head()

,y
id,
0.0,74.0
1.0,51.0
2.0,70.0
3.0,52.0
4.0,85.0


In [285]:
missing_values = X.isna().sum()
missing_values

x0       91
x1       84
x2       88
x3       91
x4       74
       ... 
x827    101
x828     95
x829     92
x830     80
x831     96
Length: 832, dtype: int64

In [286]:
#X = X.fillna(X.mean())
imputer = KNNImputer(n_neighbors=2)
X = pd.DataFrame(imputer.fit_transform(X,y))

In [287]:
X.describe().transpose().sort_values('min')

,count,mean,std,min,25%,50%,75%,max
173,1212.0,-1.836984e+22,2.502749e+22,-2.937933e+23,-2.861816e+22,-1.384906e+22,-3.738343e+21,7.002915e+22
665,1212.0,4.624869e+22,4.016835e+22,-6.392632e+22,2.722623e+22,3.942073e+22,6.067928e+22,9.935455e+23
596,1212.0,-1.796460e+14,1.260767e+14,-8.756492e+14,-2.348957e+14,-1.486501e+14,-9.590108e+13,2.926746e+14
132,1212.0,-1.385710e+06,2.094810e+05,-2.819624e+06,-1.499255e+06,-1.364324e+06,-1.279261e+06,-5.738037e+05
374,1212.0,-5.413822e+05,7.268332e+04,-1.002561e+06,-5.818485e+05,-5.337267e+05,-5.051311e+05,-1.607672e+05
...,...,...,...,...,...,...,...,...
467,1212.0,1.051675e+06,2.746376e+04,1.000249e+06,1.029032e+06,1.053533e+06,1.072575e+06,1.099984e+06
421,1212.0,1.049964e+06,2.730667e+04,1.000304e+06,1.027581e+06,1.050468e+06,1.072518e+06,1.099971e+06
637,1212.0,1.050106e+06,2.788380e+04,1.000329e+06,1.026186e+06,1.051442e+06,1.072637e+06,1.099951e+06
331,1212.0,1.048944e+06,2.837349e+04,1.000372e+06,1.024670e+06,1.050083e+06,1.070840e+06,1.099887e+06


In [288]:


model=IsolationForest(n_estimators=100,max_samples='auto',contamination=float(0.2),random_state=np.random.RandomState(42))
model.fit(X)
X['anomaly'] = model.predict(X)
y['anomaly'] = X['anomaly']
X['anomaly'].value_counts()
X = X[X.anomaly != -1]
y = y[y.anomaly != -1]

X.drop('anomaly',axis=1,inplace=True)
y.drop('anomaly',axis=1,inplace=True)


In [289]:
# selector = VarianceThreshold()
# x_train_tmp = selector.fit_transform(X,y)
selector = SelectKBest(f_regression, k=30)
selector.fit(X, y)
cols = selector.get_support(indices=True)
dataset_selected_x = X.iloc[:,cols]
dataset_selected_x


C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\feature_selection\_univariate_selection.py:306: RuntimeWarning: invalid value encountered in sqrt
  X_norms = np.sqrt(row_norms(X.T, squared=True) - n_samples * X_means**2)


,23,115,133,159,172,194,200,203,242,254,...,458,465,485,507,531,542,612,654,702,712
0,2.823225,12802.203605,3562.654007,25352.656981,2.112926,2.600569,2.468011,476.356180,1641.687814,311.969333,...,2535.339350,3459.272794,2.659421,544.374242,292.578459,1691.002260,1.995465,2.227803,25815.684394,2.265736
2,2.702694,2167.802432,3528.820216,11709.976644,1.978988,2.663890,2.293811,444.451227,1197.877859,404.289753,...,772.966759,2983.325516,2.737189,170.420569,365.810520,843.306222,1.817125,2.176732,11110.897698,2.092586
3,2.329664,12432.203273,1643.125832,22678.285676,1.658791,2.051703,1.950472,145.925347,509.564736,261.555967,...,1753.694685,3142.514697,2.043073,1466.303159,154.874824,1312.572570,1.850791,1.771077,23455.857686,1.745717
4,2.647249,6424.918043,3931.946351,16149.880881,1.941080,2.634845,2.189310,371.634319,1883.096799,335.679014,...,2277.884486,3796.544564,2.648695,942.335830,369.784157,1653.426349,1.828813,2.063448,23609.273772,2.030844
6,2.754380,23824.758898,2264.548336,30221.933846,1.871701,2.586050,2.254686,548.200270,1552.656362,278.028524,...,3405.030324,2787.103778,2.591288,2329.678657,249.862134,1716.613528,1.798282,2.101437,24317.419772,1.922731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1206,2.700417,3356.707347,3783.897507,11498.716579,1.689152,2.340502,2.005467,550.683907,1367.584527,324.547308,...,773.283181,3402.828441,2.528769,266.341600,343.740606,1245.746083,1.850791,1.862199,11875.533999,1.817313
1208,2.540225,4141.182106,3620.152732,16149.880881,1.981327,2.554647,2.190012,638.704145,1652.352245,350.545211,...,2020.712657,3820.108885,2.615226,1592.730333,317.812822,1475.566826,1.785652,2.018853,14079.097919,1.979381
1209,2.873919,2433.848973,4033.644491,8865.496693,2.097167,2.676526,2.548418,623.229623,1328.921728,426.752231,...,1302.484380,3991.612952,2.756414,297.909298,408.128414,1017.251873,1.933459,2.318883,8082.580085,2.232953
1210,2.690725,4703.596560,2870.940381,45547.152415,1.987702,2.574986,2.347072,458.980988,1388.018000,323.347622,...,3537.792852,2465.879865,2.588099,2314.121277,314.390221,1893.477664,1.915662,2.121454,42623.121583,2.095581


In [290]:
#new_X.describe()

In [291]:
#x_train_tmp = pd.DataFrame()

In [292]:
# fsel = FeatureSelector(
#         problem_type='regression',
#         featsel_runs=6,
#         keep=None,
#         n_jobs=3,
#         verbose=1)

# new_X = fsel.fit_transform(x_train_tmp, y)

In [293]:
#new_X.columns.values

In [294]:


# for j in range(832):
#         if j in accepted:
#           print (j)
#         else:
#           del dataset_selected_x[str(j)]
#dataset_selected_x = dataset_selected_x.iloc[:,cols]
# train_stats = dataset_selected_x.describe()
# train_stats = train_stats.transpose()
#dataset_selected_x

In [295]:
dataset_selected_x.to_csv('X_preprocessed.csv',index_label="id")
y.to_csv('y_preprocessed.csv',index_label="id")